# Merging RepRisk and Markit

In order to compute the ratios we need but also to be able to compare the two datasets, we need to merge them.

A first idea was to use the `isin` columns available in both dataframe but it appears that it is not well populated in the RepRisk dataset. That means we might need to match companies on their name. Doing so would require to clean the names in both datasets.

Let's first explore the idea of merging on ISIN.

In [1]:
import re
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import numpy as np
import pandas as pd
import ray
from cleanco import basename
from fuzzywuzzy import fuzz
from name_matching.name_matcher import NameMatcher
from pandasql import sqldf

import config

DATA_DIR = Path(config.DATA_DIR)
file_path = Path(DATA_DIR) / "pulled"

C:\Users\Pranav Phatak\AppData\Roaming\Python\Python38\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [35]:
ratios = pd.read_parquet(file_path/'markit_crsp_ratios.parquet')
ratios.head()

,date,cusip,isin,instrumentname,indicativefee,utilisation,shortloanquantity,quantityonloan,lendablequantity,lenderconcentration,borrowerconcentration,inventoryconcentration,shrout,short interest ratio,loan supply ratio,loan utilisation ratio,loan fee
0,2022-01-03,98956P102,US98956P1021,Zimmer Biomet Holdings Inc,0.003046,0.940322,841097.0,1529768.0,97650856.0,0.385752,0.194563,0.130224,2.089080e+08,0.007323,0.467435,0.940322,0.003046
1,2022-01-03,790148100,US7901481009,St Joe Co,0.002948,4.205225,764058.0,764058.0,15916389.0,0.306246,0.219257,0.213042,5.888300e+07,0.012976,0.270305,4.205225,0.002948
2,2022-01-03,693475105,US6934751057,Pnc Financial Services Group Inc,0.003151,1.323707,3015056.0,3419061.0,193419648.0,0.216715,0.233506,0.155540,4.226410e+08,0.008090,0.457645,1.323707,0.003151
3,2022-01-03,651639106,US6516391066,Newmont Corporation,0.003342,1.591628,5968277.0,6130376.0,340623712.0,0.121905,0.148309,0.146164,7.974350e+08,0.007688,0.427149,1.591628,0.003342
4,2022-01-03,595112103,US5951121038,Micron Technology Inc,0.001814,0.926157,3526685.0,8491586.0,501146144.0,0.272535,0.133312,0.132156,1.119777e+09,0.007583,0.447541,0.926157,0.001814


In [36]:
ratios.isna().sum()

date                            0
cusip                           0
isin                            0
instrumentname                  0
indicativefee             2518314
utilisation                661989
shortloanquantity         1817614
quantityonloan            1817614
lendablequantity           661989
lenderconcentration             0
borrowerconcentration           0
inventoryconcentration          0
shrout                    4766807
short interest ratio      4861232
loan supply ratio         4785139
loan utilisation ratio     661989
loan fee                  2518314
dtype: int64

In [38]:
test = pd.read_parquet(file_path / "merged_data.parquet")
test

,date,cusip,isin,instrumentname,indicativefee,utilisation,shortloanquantity,quantityonloan,lendablequantity,lenderconcentration,...,story_id,unsharp_incident,related_countries,related_countries_codes,severity,reach,novelty,environment,social,governance
0,2022-01-03,98956P102,US98956P1021,Zimmer Biomet Holdings Inc,0.003046,0.940322,841097.0,1529768.0,97650856.0,0.385752,...,NaN,NaN,None,None,NaN,NaN,NaN,None,None,None
1,2022-01-03,790148100,US7901481009,St Joe Co,0.002948,4.205225,764058.0,764058.0,15916389.0,0.306246,...,NaN,NaN,None,None,NaN,NaN,NaN,None,None,None
2,2022-01-03,693475105,US6934751057,Pnc Financial Services Group Inc,0.003151,1.323707,3015056.0,3419061.0,193419648.0,0.216715,...,NaN,NaN,None,None,NaN,NaN,NaN,None,None,None
3,2022-01-03,651639106,US6516391066,Newmont Corporation,0.003342,1.591628,5968277.0,6130376.0,340623712.0,0.121905,...,NaN,NaN,None,None,NaN,NaN,NaN,None,None,None
4,2022-01-03,595112103,US5951121038,Micron Technology Inc,0.001814,0.926157,3526685.0,8491586.0,501146144.0,0.272535,...,NaN,NaN,None,None,NaN,NaN,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702544,2024-02-21,82631L207,CA82631L2075,Sierra Grande Minerals Inc,NaN,NaN,700.0,700.0,NaN,0.000000,...,NaN,NaN,None,None,NaN,NaN,NaN,None,None,None
7702545,2024-02-21,22165A102,CA22165A1021,Cotec Holdings Corp,NaN,NaN,6600.0,6600.0,NaN,0.000000,...,NaN,NaN,None,None,NaN,NaN,NaN,None,None,None
7702546,2024-02-21,217641109,CA2176411098,Copperex Resources Corp,NaN,NaN,100.0,100.0,NaN,0.000000,...,NaN,NaN,None,None,NaN,NaN,NaN,None,None,None
7702547,2024-02-21,618304109,US6183041093,Morris State Bancshares Inc,NaN,NaN,100.0,100.0,NaN,0.000000,...,NaN,NaN,None,None,NaN,NaN,NaN,None,None,None


In [37]:
test.isna().sum()

date                             0
cusip                            0
isin                             0
instrumentname                   0
indicativefee              2518410
utilisation                 662084
shortloanquantity          1817615
quantityonloan             1817615
lendablequantity            662084
lenderconcentration              0
borrowerconcentration            0
inventoryconcentration           0
shrout                     4766842
short interest ratio       4861267
loan supply ratio          4785264
loan utilisation ratio      662084
loan fee                   2518410
reprisk_id                 6867290
company_name               6867290
primary_isin               6867290
current_rri                6867290
trend_rri                  6867290
peak_rri                   6867290
peak_rri_date              7136307
reprisk_rating             6867290
country_sector_average     6867290
incident_date              7693860
story_id                   7693860
unsharp_incident    

In [33]:
test.columns

Index(['date', 'cusip', 'isin', 'instrumentname', 'indicativefee',
       'utilisation', 'shortloanquantity', 'quantityonloan',
       'lendablequantity', 'lenderconcentration', 'borrowerconcentration',
       'inventoryconcentration', 'shrout', 'short interest ratio',
       'loan supply ratio', 'loan utilisation ratio', 'loan fee', 'reprisk_id',
       'company_name', 'primary_isin', 'current_rri', 'trend_rri', 'peak_rri',
       'peak_rri_date', 'reprisk_rating', 'country_sector_average',
       'incident_date', 'story_id', 'unsharp_incident', 'related_countries',
       'related_countries_codes', 'severity', 'reach', 'novelty',
       'environment', 'social', 'governance'],
      dtype='object')

In [34]:
test[['short interest ratio','loan supply ratio','loan utilisation ratio','loan fee']]

,short interest ratio,loan supply ratio,loan utilisation ratio,loan fee
0,0.007323,0.467435,0.940322,0.003046
1,0.012976,0.270305,4.205225,0.002948
2,0.008090,0.457645,1.323707,0.003151
3,0.007688,0.427149,1.591628,0.003342
4,0.007583,0.447541,0.926157,0.001814
...,...,...,...,...
7702544,NaN,NaN,NaN,NaN
7702545,NaN,NaN,NaN,NaN
7702546,NaN,NaN,NaN,NaN
7702547,NaN,NaN,NaN,NaN


In [27]:
# def count_nan(series):
#     return series.isna().sum()

# nan_count_per_date_cusip = ratios_US.groupby(['date', 'cusip']).apply(count_nan)

# cusips = list(ratios_US['cusip'].unique())

# for i in range(100):
#     print(ratios_US[ratios_US['cusip']==cusips[i]].isna().sum())

In [3]:
RepRisk_company = pd.read_parquet(file_path / "reprisk_company.parquet")
RepRisk_company.head()

,reprisk_id,company_name,primary_isin,isins
0,10,Acer Inc,TW0002353000,US0044341065 | US0044342055 | TW0002353000
1,100,Rio Tinto PLC,GB0007188757,GB0007406639 | BRRIOTBDR007 | ARDEUT112638 | G...
2,1000,Terrane Metals Corp,CA88103A1084,CA88103A1167 | US88103A3068 | CA88103A1084 | C...
3,10000,RAK Properties PJSC,AER000601016,AER000601016
4,100000,BLUECOM Co Ltd,KR7033560004,KR7033560004


In [4]:
(RepRisk_company[['primary_isin']].isna().sum() / len(RepRisk_company)).to_frame(
    "percentage_missing_isin").style.format("{:.2%}")

,percentage_missing_isin
primary_isin,85.03%


Out of more than 600k referenced companies, 85% of them have a missing ISIN in RepRisk.

Let's have a look at Markit now.

In [5]:
Markit = pd.read_parquet(file_path / "markit.parquet")
Markit_company = Markit[["isin", "instrumentname"]].drop_duplicates()
Markit_company.head()

,isin,instrumentname
0,DE0005552004,Deutsche Post Ag
1,US98956P1021,Zimmer Holdings Inc
2,US86764P1093,Sunoco Inc
3,US7901481009,St. Joe Co
4,US8265521018,Sigma-aldrich Corp


In [6]:
(Markit_company[['isin']].isna().sum() / len(Markit_company)).to_frame("percentage_missing_isin").style.format("{:.2%}")

,percentage_missing_isin
isin,5.33%


For Markit, only 5% of the companies have a missing ISIN.

We are now going to see if we can match all the available ISIN in Markit with the ones in RepRisk.

In [7]:
isin_intersection = Markit_company['isin'].dropna().isin(RepRisk_company['primary_isin'].dropna())

In [8]:
(isin_intersection.sum() / len(Markit_company['isin'].dropna())).__format__("0.2%")

'47.27%'

Only 47% of the ISIN in Markit are available in RepRisk so this is not going to work for us. We cannot merge the two datasets on ISIN only. We will have to merge on company name also.

In [9]:
RepRisk_id_on_isin = Markit_company.merge(RepRisk_company[['reprisk_id', 'primary_isin']].dropna(), left_on="isin",
                                          right_on="primary_isin", how="left")
RepRisk_id_on_isin.head()

,isin,instrumentname,reprisk_id,primary_isin
0,DE0005552004,Deutsche Post Ag,3794,DE0005552004
1,US98956P1021,Zimmer Holdings Inc,182884,US98956P1021
2,US86764P1093,Sunoco Inc,978,US86764P1093
3,US7901481009,St. Joe Co,7502,US7901481009
4,US8265521018,Sigma-aldrich Corp,7620,US8265521018


In [10]:
(RepRisk_id_on_isin[['reprisk_id']].isna().sum() / len(RepRisk_id_on_isin)).to_frame(
    "percentage_missing_reprisk_id").style.format("{:.2%}")

,percentage_missing_reprisk_id
reprisk_id,55.25%


Matching on ISIN only, we cannot match 55% of the companies in Markit with their reprisk_id in Reprisk. We will now have to look at matching companies missing isin on their name.

In [11]:
companies_missing_isin = RepRisk_id_on_isin[RepRisk_id_on_isin['reprisk_id'].isna()][['isin', 'instrumentname']]
companies_missing_isin.head()

,isin,instrumentname
14,US3371621018,First Horizon National Corp
15,US3199631041,First Data Corp
16,US2473611083,Delta Air Lines Inc
19,US0442041051,Ashland Inc
20,US1251291068,CDW COMPUTER CENTERS INC


In [12]:
RepRisk_id_on_company_name = companies_missing_isin.merge(RepRisk_company, left_on="instrumentname",
                                                          right_on="company_name", how="left")
RepRisk_id_on_company_name.head()

,isin,instrumentname,reprisk_id,company_name,primary_isin,isins
0,US3371621018,First Horizon National Corp,NaN,NaN,NaN,NaN
1,US3199631041,First Data Corp,1524305,First Data Corp,US32008D1063,US32008D1063
2,US2473611083,Delta Air Lines Inc,NaN,NaN,NaN,NaN
3,US0442041051,Ashland Inc,NaN,NaN,NaN,NaN
4,US1251291068,CDW COMPUTER CENTERS INC,NaN,NaN,NaN,NaN


In [13]:
(RepRisk_id_on_company_name[['reprisk_id']].isna().sum() / len(RepRisk_id_on_company_name)).to_frame(
    "percentage_still_missing_reprisk_id").style.format("{:.2%}")

,percentage_still_missing_reprisk_id
reprisk_id,85.13%


Merging on company name without cleaning it only allows us to match 15% of the companies without matching ISIN. We will have to clean the company names in both datasets to be able to merge them.

In [14]:
companies_still_missing_reprisk_id = RepRisk_id_on_company_name[RepRisk_id_on_company_name['reprisk_id'].isna()][
    ['isin', 'instrumentname']]
companies_still_missing_reprisk_id.head()

,isin,instrumentname
0,US3371621018,First Horizon National Corp
2,US2473611083,Delta Air Lines Inc
3,US0442041051,Ashland Inc
4,US1251291068,CDW COMPUTER CENTERS INC
5,CA4532584022,Inco Ltd


Let's summarize what we did into a single SQL request.

In [15]:
RepRisk_company_isin = RepRisk_company[['reprisk_id', 'primary_isin']].dropna()
RepRisk_company_name = RepRisk_company[['reprisk_id', 'company_name']].dropna()

match_reprisk_for_company = sqldf(""
                                  "SELECT mkc.isin, mkc.instrumentname, rrn.reprisk_id AS reprisk_id_name, rri.reprisk_id AS reprisk_id_isin "
                                  "FROM Markit_company AS mkc "
                                  "LEFT JOIN RepRisk_company_name AS rrn "
                                  "ON mkc.instrumentname = rrn.company_name "
                                  "LEFT JOIN RepRisk_company_isin AS rri "
                                  "ON mkc.isin = rri.primary_isin "
                                  )

In [16]:
match_reprisk_for_company['reprisk_id_merge'] = match_reprisk_for_company['reprisk_id_isin'].fillna(
    match_reprisk_for_company['reprisk_id_name'])

In [17]:
(1 - match_reprisk_for_company['reprisk_id_merge'].isna().sum() / len(match_reprisk_for_company)).__format__("0.2%")

'52.98%'

We are able to match 53% of the companies in Markit with their reprisk_id in Reprisk.

In [18]:
def clean_company_name(name):
    """
    Clean the company name by applying the following transformations:
    - Handle non-string inputs.
    - Convert to lowercase.
    - Remove punctuation and special characters.
    - Replace common corporate abbreviations.
    - Remove legal entity identifiers.
    - Trim whitespace.
    """
    if pd.isnull(name) or not isinstance(name, str):
        return None

    name = basename(name)
    # Convert to lowercase
    name = name.lower()
    # Remove punctuation and special characters (keep alphanumeric and spaces)
    name = re.sub(r'[^\w\s]', '', name)
    # Replace common corporate abbreviations and legal entity identifiers
    # abbreviations = {
    #     ' corporation': ' corp',
    #     ' incorporated': ' inc',
    #     ' company': ' co',
    #     ' limited': ' ltd',
    #     ' plc': '',
    #     ' llc': '',
    #     ' l p': ' lp',
    #     ' lp': ' lp'
    # }
    # for key, value in abbreviations.items():
    #     name = name.replace(key, value)
    # Trim whitespace
    name = re.sub(r'\s+', ' ', name).strip()
    return name

Again, we can update our previous sql request to match the companies on their clean names.

In [19]:
RepRisk_company_isin = RepRisk_company[['reprisk_id', 'primary_isin']].dropna()
RepRisk_company_name = RepRisk_company[['reprisk_id', 'company_name']].dropna()
Markit_company_clean = Markit_company.copy()

Markit_company_clean['cleaned_name'] = Markit_company_clean['instrumentname'].apply(clean_company_name)
RepRisk_company_name['clean_company_name'] = RepRisk_company_name['company_name'].apply(clean_company_name)

match_reprisk_for_company_clean = sqldf(""
                                        "SELECT mkc.isin, mkc.instrumentname, rrn.reprisk_id AS reprisk_id_name, rri.reprisk_id AS reprisk_id_isin, mkc.cleaned_name "
                                        "FROM Markit_company_clean AS mkc "
                                        "LEFT JOIN RepRisk_company_name AS rrn "
                                        "ON mkc.cleaned_name = rrn.clean_company_name "
                                        "LEFT JOIN RepRisk_company_isin AS rri "
                                        "ON mkc.isin = rri.primary_isin "
                                        )

In [20]:
match_reprisk_for_company_clean['reprisk_id_merge'] = match_reprisk_for_company_clean['reprisk_id_isin'].fillna(
    match_reprisk_for_company_clean['reprisk_id_name'])

In [21]:
(1 - match_reprisk_for_company_clean['reprisk_id_merge'].isna().sum() / len(
    match_reprisk_for_company_clean)).__format__("0.2%")

'59.56%'

In [22]:
still_missing2 = match_reprisk_for_company_clean[match_reprisk_for_company_clean['reprisk_id_merge'].isna()][
    ['isin', 'instrumentname', 'cleaned_name']]

In [23]:
RepRisk_company_name_unique_index = RepRisk_company_name.reset_index(drop=True, inplace=False)

In [26]:
ray.shutdown()
ray.init()

matcher = NameMatcher(ngrams=(2, 5),
                      top_n=10,
                      number_of_rows=500,
                      number_of_matches=3,
                      lowercase=True,
                      punctuations=True,
                      remove_ascii=True,
                      legal_suffixes=False,
                      common_words=False,
                      preprocess_split=False,
                      verbose=False)

matcher.set_distance_metrics(['iterative_sub_string', 'pearson_ii', 'bag', 'fuzzy_wuzzy_partial_string', 'editex'])

matcher.load_and_process_master_data(column='clean_company_name',
                                     df_matching_data=RepRisk_company_name_unique_index,
                                     transform=True)

@ray.remote
def match_name_parallel(adjusted_names, matcher):
    results = matcher.match_names(to_be_matched=adjusted_names, column_matching='cleaned_name')
    return results

results = []
for i in range(0, len(still_missing2), 100):
    results.append(match_name_parallel.remote(still_missing2[i:i + 100], matcher))

matches = pd.concat(ray.get(results))

2024-02-25 14:26:55,056	INFO worker.py:1724 -- Started a local Ray instance.
(raylet) Spilled 2595 MiB, 4 objects, write throughput 765 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 4542 MiB, 7 objects, write throughput 855 MiB/s.
(raylet) Spilled 9085 MiB, 14 objects, write throughput 1065 MiB/s.
(raylet) Spilled 16872 MiB, 26 objects, write throughput 1107 MiB/s.
(raylet) Spilled 33096 MiB, 51 objects, write throughput 1194 MiB/s.
(raylet) Spilled 65543 MiB, 101 objects, write throughput 1164 MiB/s.


In [27]:
matches.merge(RepRisk_company_name_unique_index, left_on="match_index_0", right_index=True, how="left")

,original_name,match_name_0,score_0,match_index_0,match_name_1,score_1,match_index_1,match_name_2,score_2,match_index_2,reprisk_id,company_name,clean_company_name
17,first horizon national,first horizon,79.052839,485285,horizon,59.262131,313922,horizon,59.262131,583632,2693,first horizon,first horizon
20,delta air lines,delta air lines inc delta,79.161865,496055,air link,63.954512,198577,delta,63.748140,548135,2978,delta air lines inc delta,delta air lines inc delta
23,ashland,mushland,82.192777,257793,dashlane,81.439926,611530,ashland farm,77.699523,321725,200391,mushland pty ltd,mushland
24,cdw computer centers,entre computer centers,83.211100,48113,rd computer co,71.645984,540479,at computer,63.027683,475723,1181698,entre computer centers inc,entre computer centers
44,scheringplough,slough,63.009746,366287,clough,60.581115,85153,ring,56.399262,420396,2430935,slough ltd,slough
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77332,moon river capital,river capital,85.960993,370126,capital,66.119039,512101,iron river,61.440307,319720,2435360,river capital inc,river capital
77336,refined metals,red metal,77.149540,580624,red metal,77.149540,161158,meta,54.970691,535120,55691,red metal ltd,red metal
77337,atlasclear holdings,atlas,61.571510,586749,atlas,61.571510,415918,atlas,61.571510,20504,57233,atlas sa,atlas
77339,semilux,milux,83.420407,519438,semi,76.017944,100013,filux,68.548585,571496,38707,milux corp bhd,milux


In [25]:
reprisk_df = RepRisk_company_name.copy()

# Create a set of unique cleaned names from the reprisk_df for faster lookup
unique_cleaned_names = set(reprisk_df['clean_company_name'].dropna())

# Create a dictionary to cache the results of fuzzy matching
fuzzy_match_cache = defaultdict(lambda: None)


def get_reprisk_id_optimized(still_missing_name, threshold=97):
    """
    Optimized function to use fuzzy matching to find the closest match in the reprisk dataframe
    for a given company name. It uses caching to speed up the process.
    """
    # Check if the name is already in the cache
    if still_missing_name in fuzzy_match_cache:
        return fuzzy_match_cache[still_missing_name]

    if still_missing_name is None:
        return None

    # Find the best match for the company name in the set of unique cleaned names
    best_match = None
    best_score = 0
    for candidate in unique_cleaned_names:
        score = fuzz.partial_ratio(still_missing_name, candidate)
        if score > best_score:
            best_score = score
            best_match = candidate
        # If we reach a score that's good enough, we can stop searching
        if score >= threshold:
            break

    # If the best score is above the threshold, find the reprisk_id from the dataframe
    if best_score >= threshold:
        reprisk_id = reprisk_df[reprisk_df['clean_company_name'] == best_match]['reprisk_id'].values[0]
    else:
        reprisk_id = None

    # Cache the result
    fuzzy_match_cache[still_missing_name] = reprisk_id
    return reprisk_id

In [26]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    with ThreadPoolExecutor(max_workers=n_cores) as executor:
        df = pd.concat(executor.map(func, df_split))
    return df


def apply_func_to_series(data_series):
    return data_series.apply(get_reprisk_id_optimized)

# subset_still_missing_df = still_missing2.copy()
# subset_still_missing_df = parallelize_dataframe(subset_still_missing_df['cleaned_name'], apply_func_to_series, n_cores=4)
# subset_still_missing_df

In [27]:
subset_still_missing_df = pd.read_csv("subset_still_missing_df.csv", index_col=0)

In [28]:
match_fuzzy = still_missing2.join(subset_still_missing_df, rsuffix="_reprisk_id")

In [29]:
match_fuzzy.rename(
    columns={"isin": "isin_markit", "instrumentname": "instrumentname_markit", "cleaned_name": "cleaned_name_markit",
             "cleaned_name_reprisk_id": "reprisk_id_matched"}, inplace=True)

In [30]:
# fuzzy_match_cache_df = pd.DataFrame(fuzzy_match_cache.items(), columns=['cleaned_name', 'reprisk_id'])

In [31]:
fuzzy_match_cache_df = pd.read_csv("fuzzy_match_cache.csv", index_col=0)

In [ ]:
match_reprisk_for_company_fuzzy = sqldf(""
                                        "SELECT mf.isin_markit, mf.cleaned_name_markit, mf.instrumentname_markit, mf.reprisk_id_matched, rrn.reprisk_id AS reprisk_id_check, rrn.clean_company_name AS cleaned_name_check "
                                        "FROM match_fuzzy AS mf "
                                        "LEFT JOIN RepRisk_company_name AS rrn "
                                        "ON mf.reprisk_id_matched = rrn.reprisk_id "
                                        )

In [35]:
# run fuzzy to get the score for each line between clean markit name and chack name clean
# match_reprisk_for_company_fuzzy['fuzzy_score'] = match_reprisk_for_company_fuzzy.apply(lambda x: fuzz.partial_ratio(x['cleaned_name_markit'], x['cleaned_name_check']), axis=1)

In [32]:
match_reprisk_for_company_fuzzy = pd.read_csv("match_reprisk_for_company_fuzzy.csv", index_col=0)

In [33]:
match_reprisk_for_company_fuzzy

,isin_markit,cleaned_name_markit,instrumentname_markit,reprisk_id_matched,reprisk_id_check,cleaned_name_check,fuzzy_score
0,US3371621018,first horizon national corp,First Horizon National Corp,1422034.0,1422034.0,ati,100
1,US2473611083,delta air lines inc,Delta Air Lines Inc,2155238.0,2155238.0,inc,100
2,US0442041051,ashland inc,Ashland Inc,3117.0,3117.0,ashland inc formerly ashland global holdings i...,100
3,US1251291068,cdw computer centers inc,CDW COMPUTER CENTERS INC,101758.0,101758.0,cdw,100
4,CA4532584022,inco ltd,Inco Ltd,186366.0,186366.0,renopharma inc ltd,93
...,...,...,...,...,...,...,...
32222,CA97536W2076,winshear gold corp,Winshear Gold Corp,1786087.0,1786087.0,ld,100
32223,US8200141088,sharplink gaming ltd,Sharplink Gaming Ltd,1514772.0,1514772.0,link,100
32224,CA00792K1075,aero energy ltd,Aero Energy Ltd,781678.0,781678.0,senergy ltd,91
32225,US87975F1049,telomir pharmaceuticals inc,Telomir Pharmaceuticals Inc,2251983.0,2251983.0,epi pharmaceuticals inc,91


In [43]:
markit2022_ = pd.read_parquet(file_path / "markit2022_.parquet")

In [44]:
markit2022_

,datadate,cusip,shrout,isin,instrumentname,indicativefee,utilisation,shortloanquantity,quantityonloan,lendablequantity,lenderconcentration,borrowerconcentration,inventoryconcentration
0,2022-01-03,00032Q10,20895.0,US00032Q1040,Aadi Bioscience Inc,0.017219,36.346460,386616.0,386616.0,1005248.0,0.271643,0.206404,0.228618
1,2022-01-03,00036110,35469.0,US0003611052,Aar Corp,0.002761,3.703486,839643.0,886008.0,18960896.0,0.240280,0.225955,0.168525
2,2022-01-03,00164V10,30892.0,US00164V1035,Amc Networks Inc,0.002920,25.957410,4138402.0,4139902.0,14484817.0,0.327368,0.224092,0.182572
3,2022-01-03,00216W20,207283.0,US00216W1099,Ati Physical Therapy Inc,0.017493,53.313670,3853695.0,3856695.0,6073751.0,0.198212,0.519702,0.183514
4,2022-01-03,00287Y10,1767880.0,US00287Y1091,Abbvie Inc,0.003256,0.261216,1574813.0,2000571.0,557075584.0,0.140679,0.143139,0.132769
...,...,...,...,...,...,...,...,...,...,...,...,...,...
228485,2023-12-29,Y2711Y11,11052.0,SG9999014831,Giga Media Ltd,0.137748,6.405102,7800.0,7800.0,81702.0,1.000000,1.000000,0.454410
228486,2023-12-29,Y3894J18,23615.0,MHY3894J1872,Imperial Petroleum Inc,0.115836,93.988070,3798367.0,3798367.0,2093563.0,0.743503,0.338524,0.683525
228487,2023-12-29,Y3894J18,23615.0,MHY3894J1120,Imperial 8 75 Srs A Cum Red Prep Prf,NaN,0.000000,NaN,NaN,477.0,0.000000,0.000000,0.414445
228488,2023-12-29,Y6430L20,3735.0,MHY6430L2029,Oceanpal Inc,0.442730,1.708853,1810.0,1810.0,84368.0,0.836157,0.434993,0.803737


In [39]:
markit = pd.read_parquet(file_path / "markit.parquet")

In [40]:
markit[markit.datadate >= "2022-01-01"]

,datadate,dxlid,isin,sedol,cusip,instrumentname,indicativefee,utilisation,shortloanquantity,quantityonloan,lendablequantity,lenderconcentration,borrowerconcentration,inventoryconcentration
0,2022-01-03,DX00000021,US98956P1021,2783815,98956P102,Zimmer Biomet Holdings Inc,0.003046,0.940322,841097.0,1529768.0,97650856.0,0.385752,0.194563,0.130224
1,2022-01-03,DX00000023,US7901481009,2768663,790148100,St Joe Co,0.002948,4.205225,764058.0,764058.0,15916389.0,0.306246,0.219257,0.213042
2,2022-01-03,DX00000026,US6934751057,2692665,693475105,Pnc Financial Services Group Inc,0.003151,1.323707,3015056.0,3419061.0,193419648.0,0.216715,0.233506,0.155540
3,2022-01-03,DX00000029,US6516391066,2636607,651639106,Newmont Corporation,0.003342,1.591628,5968277.0,6130376.0,340623712.0,0.121905,0.148309,0.146164
4,2022-01-03,DX00000030,US5951121038,2588184,595112103,Micron Technology Inc,0.001814,0.926157,3526685.0,8491586.0,501146144.0,0.272535,0.133312,0.132156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457298,2024-02-16,DX02521070,CA92847V5018,BNC4594,92847V501,Vitalhub Corp,NaN,0.000000,NaN,NaN,2400.0,0.000000,0.000000,1.000000
457299,2024-02-16,DX02521083,BRVIVRD02M16,BRPS2Y5,P9T02N498,Viver Incorporadora E Const Rts,NaN,NaN,12296.0,12296.0,NaN,0.000000,1.000000,0.000000
457300,2024-02-16,DX02521177,CA98422W2022,BN7R811,98422W202,Xigem Technologies Corp,NaN,NaN,300.0,300.0,NaN,0.000000,1.000000,0.000000
457301,2024-02-16,DX02521184,CA6154501036,BK95TF3,615450103,Moon River Capital Ltd,NaN,NaN,600.0,600.0,NaN,0.000000,1.000000,0.000000


In [41]:
crsp = pd.read_parquet(file_path / "crsp.parquet")

In [42]:
crsp.cusip.nunique()

17853